# Analysis of Global Warming Tweets on January 2023.
This proejct is based on global warming tweets tweeted on January 2023. I have added emotion (anger, joy, opotimism and sadness) and gender at the end of each tweets. Emotion analysis was performed using a pre-trained model from Hugging Face (Twitter-roBERTa-base for Emotion Recognition). This is a roBERTa-base model trained on ~58M tweets and finetuned for emotion recognition with the TweetEval benchmark. Each tweet is classified into four emotions (joy, optimism, anger, and sadness) with a confidence score. In addition, gender is extracted based on first name of user account if a user account has a real first name and the gender can be identified by python package gender guesser. 

# Read tweets into a data frame

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from helper_functions import displayByGroup
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# check if the Spark session is active. If it is activate, close it

try:
    if spark:
        spark.stop()
except:
    pass    

spark = (SparkSession.builder.appName("Multidimensional Data Frame")
        .config("spark.port.maxRetries", "100")
        .config("spark.sql.mapKeyDedupPolicy", "LAST_WIN")  # This configuration allow the duplicate keys in the map data type.
#        .config("spark.driver.memory", "16g")
        .getOrCreate())

# confiture the log level (defaulty is WWARN)
spark.sparkContext.setLogLevel('ERROR')

# read the global warming tweets

df=spark.read.parquet('/opt/shared/globalwarming_202301')

# In class exercise

## Extract mentions of each tweet account received.

In [ ]:
#df.select('author').printSchema()

In [ ]:
df.select('author.entities.description.mentions').printSchema()

In [ ]:
from pyspark.sql.functions import col, explode, desc
df1=df.select('author.username', col('author.entities.description.mentions.username').alias('mentions')).filter(F.col('mentions').isNotNull())

df2=df1.select('username', explode('mentions').alias('mentions'))

In [ ]:
# users who received most mentions

df2.groupBy('mentions').count().orderBy(desc('count')).show()

In [ ]:
# user who mentioned most people

df2.groupBy('username').count().orderBy(desc('count')).show()

## extract user location from tweets

## Extract hashtags from tweets

## Extract Entity (place, person, organizations) from tweets